In [0]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import concatenate
import numpy as np
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [0]:
def generate_sequence(length, n_unique):
	return [randint(1, n_unique-1) for _ in range(length)]

In [0]:
def get_dataset(n_in, n_out, cardinality, n_samples):
        X1, X2, y = list(), list(), list()
        for _ in range(n_samples):
            # generate source sequence
            source = generate_sequence(n_in, (cardinality-1)/2)
            # define padded target sequence
            rev = source[:n_out]
            rev.reverse()
            target = [r for r in rev]
            target.extend(2*r for r in source[:n_out])
            # create padded input target sequence
            target_in = [0] + target[:2]
            target_in.extend([0]+target[3:5])
            # encode
            src_encoded = to_categorical([source], num_classes=cardinality)
            tar_encoded = to_categorical([target], num_classes=cardinality)
            tar2_encoded = to_categorical([target_in], num_classes=cardinality)
            # store
            X1.append(src_encoded)
            X2.append(tar2_encoded)
            y.append(tar_encoded)
        X1 = np.squeeze(array(X1), axis=1) 
        X2 = np.squeeze(array(X2), axis=1) 
        y = np.squeeze(array(y), axis=1) 
        return array(X1), array(X2), array(y)

In [0]:
def define_models(n_input, n_output, n_units):
        # define training encoder for reverse
        encoder_inputs = Input(shape=(None, n_input))
        encoder = LSTM(n_units, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        encoder_states = [state_h, state_c]
  

        # define training decoder for reverse
        decoder_inputs_rev = Input(shape=(None, n_output))
        decoder_lstm_rev = LSTM(n_units, return_sequences=True, return_state=True)
        decoder_outputs_rev, _, _ = decoder_lstm_rev(decoder_inputs_rev, initial_state=encoder_states)
        decoder_dense_rev = Dense(n_output, activation='softmax')
        decoder_outputs_rev = decoder_dense_rev(decoder_outputs_rev)

        # define the traning decoder for multiply
        decoder_inputs_mult = Input(shape=(None, n_output))
        decoder_lstm_mult = LSTM(n_units, return_sequences=True, return_state=True)
        decoder_outputs_mult, _, _ = decoder_lstm_mult(decoder_inputs_mult, initial_state=encoder_states)
        decoder_dense_mult= Dense(n_output, activation='softmax')
        decoder_outputs_mult = decoder_dense_mult(decoder_outputs_mult)


        model = Model([encoder_inputs, decoder_inputs_rev, decoder_inputs_mult], [decoder_outputs_rev, decoder_outputs_mult])
        # define training decoder for multiply

        # define inference encoder
        encoder_model = Model(encoder_inputs, encoder_states)
        # define inference decoder reverse
        decoder_state_input_h_rev = Input(shape=(n_units,))
        decoder_state_input_c_rev = Input(shape=(n_units,))
        decoder_states_inputs_rev = [decoder_state_input_h_rev, decoder_state_input_c_rev]
        decoder_outputs_rev, state_h_rev, state_c_rev = decoder_lstm_rev(decoder_inputs_rev, initial_state=decoder_states_inputs_rev)
        decoder_states_rev = [state_h_rev, state_c_rev]
        decoder_outputs_rev = decoder_dense_rev(decoder_outputs_rev)
        decoder_model_rev = Model([decoder_inputs_rev] + decoder_states_inputs_rev, [decoder_outputs_rev] + decoder_states_rev)

        # define inference decoder mult
        decoder_state_input_h_mult = Input(shape=(n_units,))
        decoder_state_input_c_mult = Input(shape=(n_units,))
        decoder_states_inputs_mult = [decoder_state_input_h_mult, decoder_state_input_c_mult]
        decoder_outputs_mult, state_h_mult, state_c_mult = decoder_lstm_mult(decoder_inputs_mult, initial_state=decoder_states_inputs_mult)
        decoder_states_mult = [state_h_mult, state_c_mult]
        decoder_outputs_mult = decoder_dense_mult(decoder_outputs_mult)
        decoder_model_mult = Model([decoder_inputs_mult] + decoder_states_inputs_mult, [decoder_outputs_mult] + decoder_states_mult)
        # return all models
        return model, encoder_model, decoder_model_rev, decoder_model_mult

In [0]:
# generate target given source sequence
def predict_sequence(infenc, infdec_rev, infdec_mult, source, n_steps, cardinality):
        # encode
        state = infenc.predict(source)
        # start of sequence input
        target_seq_rev = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
        target_seq_mult = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
        # collect predictions
        output = list()
        for t in range(3):
            # predict next char
            yhat_rev, h_rev, c_rev = infdec_rev.predict([target_seq_rev] + state)
            # store prediction
            output.appensd(yhat_rev[0,0,:])
            # update state
            state = [h_rev, c_rev]
            # update target sequence
            target_seq_rev = yhat_rev
        state = infenc.predict(source)
        for t in range(3):
            # predict next char
            yhat_mult, h_mult, c_mult = infdec_mult.predict([target_seq_mult] + state)
            # store prediction
            output.append(yhat_mult[0,0,:])
            # update state
            state = [h_mult, c_mult]
            # update target sequence
            target_seq_mult = yhat_mult
        return array(output)

In [0]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

In [0]:
# configure problem
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3
# define model
train, infenc, infdec_rev, infdec_mult = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
print(X1.shape,X2.shape,y.shape)
# train model
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
train.fit([X1, X2[:, :3, :], X2[:, 3:, :]], [y[:, :3, :],y[:, 3:, :]], epochs=5, verbose = 1, callbacks=[earlystop])

W0725 15:36:42.594272 140439306545024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 15:36:42.632059 140439306545024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 15:36:42.639251 140439306545024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0725 15:36:43.570635 140439306545024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0725 15:36:43.595568 140439306545024 deprecation_wrappe

(100000, 6, 51) (100000, 6, 51) (100000, 6, 51)


W0725 15:36:48.607455 140439306545024 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0725 15:36:50.392460 140439306545024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
100000/100000 [==============================] - 80s 796us/step - loss: 0.9095 - dense_1_loss: 0.5088 - dense_2_loss: 0.4008 - dense_1_acc: 0.8266 - dense_2_acc: 0.8827
Epoch 2/5
   224/100000 [..............................] - ETA: 1:16 - loss: 0.0358 - dense_1_loss: 0.0266 - dense_2_loss: 0.0093 - dense_1_acc: 0.9940 - dense_2_acc: 1.0000

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,dense_1_loss,dense_2_loss,dense_1_acc,dense_2_acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


100000/100000 [==============================] - 75s 746us/step - loss: 0.0160 - dense_1_loss: 0.0102 - dense_2_loss: 0.0059 - dense_1_acc: 0.9987 - dense_2_acc: 0.9995
Epoch 3/5
100000/100000 [==============================] - 75s 748us/step - loss: 0.0065 - dense_1_loss: 0.0038 - dense_2_loss: 0.0027 - dense_1_acc: 0.9993 - dense_2_acc: 0.9996
Epoch 4/5
100000/100000 [==============================] - 75s 746us/step - loss: 0.0053 - dense_1_loss: 0.0026 - dense_2_loss: 0.0027 - dense_1_acc: 0.9994 - dense_2_acc: 0.9993
Epoch 5/5
100000/100000 [==============================] - 75s 746us/step - loss: 0.0020 - dense_1_loss: 0.0012 - dense_2_loss: 7.3810e-04 - dense_1_acc: 0.9998 - dense_2_acc: 0.9999
